In [1]:
import os
import tempfile

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

plt.style.use('seaborn-whitegrid')

2023-03-28 17:02:18.731736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_634947/2468769046.py:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
from pymongo import MongoClient
import pandas as pd

mongo = MongoClient()
data = list(mongo["data_4"]["events"].find({},{"_id": 0}))
movies = list(mongo["data_4"]["data"].find({},{"_id": 0, "tag": 0}))
df_meta = pd.DataFrame(movies)
df = pd.DataFrame(data)
print(df.columns)
BEHAVIORS = {
    "apply": 5.0,
    "search": 4.0,
    "like": 4.0,
    "view": 3.0,
    "ignore": 1.0,
    "dislike": 0.0,
    "hated": 0.0,
}

def create_rating(behavior):
    return BEHAVIORS.get(behavior, 0)

def create_skill_text(lst_skill):
    return ','.join(lst_skill)

df_meta["skill_text"] = df_meta.skill.apply(create_skill_text)
    
df["user_rating"] = df.behavior.apply(create_rating)
ratings = df[["userid", "jobId","user_rating"]].merge(df_meta, on="jobId", how="left")

tensor_slices = {
    "user_id": ratings.userid.values.tolist(),
    "job_id": ratings.jobId.values.tolist(),
    "location": ratings.location.values.tolist(),
    "category": ratings.category.values.tolist(),
    "level": ratings.level.values.tolist(),
    "job_title": ratings.title.values.tolist(),
    "skill_text": ratings.skill_text.values.tolist()
}

movies = tf.data.Dataset.from_tensor_slices({
    "job_id": df_meta.jobId.values.tolist(),
    "category": df_meta.category.values.tolist(),
    "location": df_meta.location.values.tolist(),
    "level": df_meta.level.values.tolist(),
    "job_title": df_meta.title.values.tolist(),
    "skill_text": df_meta.skill_text.values.tolist()
})
ratings = tf.data.Dataset.from_tensor_slices(tensor_slices)

Index(['userid', 'jobId', 'job', 'skill', 'level', 'location', 'salary',
       'behavior', 'time'],
      dtype='object')


2023-03-28 17:02:22.704783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 17:02:22.725259: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 17:02:22.725414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 17:02:22.725835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [3]:
for i in ratings.take(1):
    print(i)

{'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'User_Senior_Data_Scientist_HCM_43'>, 'job_id': <tf.Tensor: shape=(), dtype=string, numpy=b'Job_Leader_Data_Scientist_other_Leader_Q1GEHW'>, 'location': <tf.Tensor: shape=(), dtype=string, numpy=b'other'>, 'category': <tf.Tensor: shape=(), dtype=string, numpy=b'Data Scientist'>, 'level': <tf.Tensor: shape=(), dtype=string, numpy=b'Leader'>, 'job_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Leader Data Scientist'>, 'skill_text': <tf.Tensor: shape=(), dtype=string, numpy=b'Python,Deep Learning,Big-data,Machine Learning,Python'>}


In [4]:
from functools import reduce


unique_job_titles = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["job_title"]))))
unique_job_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["job_id"]))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))
unique_locations = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["location"]))))
unique_levels = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["level"]))))
unique_skills = np.unique(reduce(lambda x, y: x + y, df_meta.skill.values.tolist()))
job_title_vocabs = np.unique(reduce(lambda x, y: x + y, df_meta.title.str.split(" ").tolist()))
unique_category = np.unique(df_meta.category.values.tolist())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
unique_category

array(['Backend Developer', 'Big Data Engineer', 'Data Analyst',
       'Data Engineer', 'Data Scientist', 'Devops Engineer',
       'Front-end Developer', 'Fullstack Developer', 'Java Engineer',
       'Machine Learning', 'NodeJS Developer', 'Python Developer'],
      dtype='<U19')

In [6]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"])
    ], axis=1)

In [7]:
for i in ratings.take(1):
    print(i)

UserModel()({"user_id": ["User_Senior_Data_DN_15"]})

{'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'User_Senior_Data_Scientist_HCM_43'>, 'job_id': <tf.Tensor: shape=(), dtype=string, numpy=b'Job_Leader_Data_Scientist_other_Leader_Q1GEHW'>, 'location': <tf.Tensor: shape=(), dtype=string, numpy=b'other'>, 'category': <tf.Tensor: shape=(), dtype=string, numpy=b'Data Scientist'>, 'level': <tf.Tensor: shape=(), dtype=string, numpy=b'Leader'>, 'job_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Leader Data Scientist'>, 'skill_text': <tf.Tensor: shape=(), dtype=string, numpy=b'Python,Deep Learning,Big-data,Machine Learning,Python'>}


<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.02236619,  0.04711631, -0.03581182, -0.0390179 ,  0.01522123,
         0.02411312, -0.01010716,  0.02359494, -0.04594259, -0.02136555,
        -0.01033449,  0.02416765, -0.03813561, -0.04417056, -0.00364971,
        -0.02353035,  0.00146408,  0.03224584, -0.02657051,  0.04053435,
        -0.02815633,  0.0333264 ,  0.01613083, -0.01159348,  0.01468903,
         0.01689574, -0.01371038, -0.0030042 ,  0.03742046,  0.04632069,
        -0.0307781 , -0.01746657]], dtype=float32)>

In [8]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    print(feature_embedding)
    return self.dense_layers(feature_embedding)

In [9]:
QueryModel([32])({"user_id": ["User_Senior_Data_DN_15"]})

tf.Tensor(
[[ 0.0356055  -0.00985302 -0.01391516 -0.00711821  0.00623367 -0.04315417
   0.04078274 -0.00969734 -0.01865138 -0.02708879  0.0051927   0.01862014
  -0.02774283  0.01440542 -0.00386498 -0.04016288 -0.0056999  -0.02242583
   0.00318179  0.02644986 -0.00109179  0.00601713 -0.0121747   0.02007195
  -0.03249978  0.04657267  0.03774026 -0.02799339  0.03742131 -0.01231729
  -0.04372475  0.02282113]], shape=(1, 32), dtype=float32)


<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-1.0284461e-02, -2.4050713e-02,  4.5108549e-02, -4.8676338e-03,
         3.3449236e-02, -1.4045844e-03, -2.0982698e-05, -3.9254420e-02,
         2.1986797e-02,  1.3038442e-03,  1.9355714e-02, -2.0266697e-03,
         2.2263005e-02,  1.6035080e-02,  3.4869134e-02,  3.1468719e-02,
         2.1573436e-02, -6.7312368e-03, -8.6092614e-03,  2.7431104e-02,
         1.8572111e-02,  1.6591094e-02, -6.3403092e-02,  6.2853947e-02,
        -2.8002774e-02, -1.7819507e-02,  1.0836715e-02, -2.0060360e-02,
         3.9487132e-03,  1.0313468e-02, -2.0944122e-02, -1.9162279e-03]],
      dtype=float32)>

In [10]:
def tokenization(t):
    return tf.strings.split(t, ',')

def tokenization_1(t):
    return tf.strings.split(t, ' ')


class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.job_id_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_job_ids,mask_token=None),
      tf.keras.layers.Embedding(len(unique_job_ids) + 1, 32)
    ])
    self.location_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_locations,mask_token=None),
      tf.keras.layers.Embedding(len(unique_locations) + 1, 32)
    ])
    self.level_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_levels,mask_token=None),
      tf.keras.layers.Embedding(len(unique_levels) + 1, 32)
    ])
    self.category_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_category,mask_token=None),
      tf.keras.layers.Embedding(len(unique_category) + 1, 32)
    ])
    self.skill_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(
          max_tokens=50,
          standardize=None,
          split=tokenization,
          vocabulary=unique_skills,
          pad_to_max_tokens=True
      ),
      tf.keras.layers.Embedding(len(unique_skills) + 1, 32),
      tf.keras.layers.GlobalAvgPool1D()
    ])
    
#     self.job_title_embedding = tf.keras.Sequential([
#       tf.keras.layers.TextVectorization(
#           max_tokens=50,
#           standardize=None,
#           split=tokenization_1,
#           vocabulary=job_title_vocabs,
#           pad_to_max_tokens=True
#       ),
#       tf.keras.layers.Embedding(len(job_title_vocabs) + 1, 32),
#       tf.keras.layers.GlobalAvgPool1D()
#     ])

  def call(self, features):
    return tf.concat([
        self.job_id_embedding(features["job_id"]),
        self.category_embedding(features["category"]),
        self.location_embedding(features["location"]),
        self.level_embedding(features["level"]),
        self.skill_embedding(features["skill_text"])
    ], axis=1)

In [11]:
MovieModel()({"job_id": ["Job_Senior_Machine_Learning_other_Senior_79Z5AL"], 
              "job_title": ["Senior Machine Learning"],
              "category": ["Machine Learning"],
              "location": ["other"],
              "level": ["Senior"],
              "skill_text": ["MLops,Machine Learning,Python,Deep Learning,Machine Learning"]})

<tf.Tensor: shape=(1, 160), dtype=float32, numpy=
array([[-0.01673947,  0.01867235,  0.00918338,  0.04249363, -0.02767295,
         0.02271017, -0.00895195, -0.04118539, -0.00764239,  0.00698882,
        -0.0163398 , -0.04421039,  0.01781925,  0.0381946 , -0.04996772,
        -0.02346647, -0.04742343, -0.03186274,  0.02298388, -0.03290059,
        -0.02073327, -0.02836282,  0.04109964,  0.0288066 , -0.00976581,
         0.01723843, -0.0269614 , -0.04497306,  0.01741442, -0.03940226,
         0.02924262, -0.02721366, -0.01376605,  0.01498267,  0.04032034,
        -0.02647961, -0.00776209,  0.02970942,  0.03140192,  0.03578639,
         0.00156541,  0.04211773, -0.00103685, -0.02161527,  0.02784734,
         0.03679811,  0.03650156, -0.04967643,  0.02396234, -0.03710496,
         0.02542565,  0.02202166,  0.03019777,  0.02390262,  0.00270981,
         0.00489514,  0.03018141, -0.04779547,  0.02434241, -0.00886893,
        -0.01313818,  0.03012142,  0.03978567,  0.03937488, -0.03038119,
 

In [12]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [13]:
CandidateModel([32])({"job_id": ["Job_Senior_Machine_Learning_other_Senior_79Z5AL"], 
              "job_title": ["Senior Machine Learning"],
              "location": ["other"],
              "category": ["Machine Learning"],
              "level": ["Senior"],
              "skill_text": ["MLops,Machine Learning,Python,Deep Learning,Machine Learning"]})

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 2.9200973e-02, -5.4408036e-02,  3.5864998e-02, -1.4224683e-02,
         1.5656676e-02,  5.2559022e-03,  2.2594284e-02, -2.9829346e-02,
         2.5601985e-03, -3.8422465e-02, -4.4018924e-03, -4.7970422e-02,
        -8.1535717e-03,  7.8037344e-02, -5.5568419e-02, -7.0533026e-03,
        -4.4866469e-02,  8.5246451e-03,  8.5763820e-03, -5.0382894e-02,
         1.1462960e-03,  6.3596759e-05,  7.1259111e-02,  4.4505633e-02,
         1.9666661e-02, -1.9976575e-02, -4.9168661e-02, -3.5677217e-02,
         1.2695152e-02,  1.6742125e-02, -5.0298609e-02, -4.8257202e-02]],
      dtype=float32)>

In [14]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"]
    })
    movie_embeddings = self.candidate_model({
        "job_id": features["job_id"],
        "job_title": features["job_title"],
        "category": features["category"],
        "location": features["location"],
        "skill_text": features["skill_text"],
        "level": features["level"],
    })

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

In [15]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [16]:
num_epochs = 300

model = MovielensModel([128, 64, 32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

Tensor("query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)


2023-03-28 17:02:51.482129: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x201a82e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-28 17:02:51.482150: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-03-28 17:02:51.485074: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-28 17:02:51.525124: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:51.550070: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster u

2023-03-28 17:02:53.422064: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:53.499157: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:53.579924: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-28 17:02:55.425230: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:55.501713: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:55.588390: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-28 17:02:58.106945: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:58.193661: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:02:58.286980: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-28 17:03:01.697653: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:03:02.213519: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:03:02.436888: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-28 17:03:09.489504: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:03:10.257734: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:03:10.801103: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-28 17:03:34.214800: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:03:35.063003: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-28 17:03:39.165141: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

In [17]:
# one_layer_history.history["factorized_top_k/top_100_categorical_accuracy"]

In [18]:
# model.query_model({"user_id": np.array(["User_Senior_Data_DN_15"])})

index = tfrs.layers.factorized_top_k.BruteForce(model.query_model, k=70)
index.index_from_dataset(
    movies.batch(100).map(lambda x: (x["job_id"], model.candidate_model(x))))

In [19]:
score, pred = index({"user_id": np.array(["User_Fresher_Backend_Developer_DN_0"])})
print(pred.numpy())
score

tf.Tensor(
[[ 0.18641126 -0.72842294  0.06562594 -0.13979457  0.44185242  0.5221946
  -0.19198662 -0.4325505   0.0956023   0.26997694  0.35297924 -0.11840552
  -0.02365739  0.2474908  -0.17326188 -0.50323457  0.03224694 -0.30025646
  -0.06519936 -0.14000528 -0.13099912 -0.9593323  -0.3446247   0.22645451
  -0.02653072  0.6399289  -0.42444944  0.4719108   0.07561869  0.25160024
  -0.21619645 -0.10724824]], shape=(1, 32), dtype=float32)
[[b'Job_Fresher_Backend_Developer_DN_Fresher_XVUJ3U'
  b'Job_Fresher_Backend_Developer_other_Fresher_EHNOUV'
  b'Job_Fresher_Backend_Developer_DN_Fresher_9BRW2M'
  b'Job_Fresher_Backend_Developer_DN_Fresher_5NKI9W'
  b'Job_Fresher_Backend_Developer_other_Fresher_S0TCVR'
  b'Job_Middle_Fullstack_Developer_DN_Middle_85ZOH7'
  b'Job_Junior_Backend_Developer_DN_Junior_EBI79X'
  b'Job_Fresher_Backend_Developer_HCM_Fresher_SFSOBQ'
  b'Job_Fresher_Backend_Developer_HN_Fresher_Z41FJQ'
  b'Job_Fresher_Backend_Developer_HCM_Fresher_GLU5YO'
  b'Job_Junior_Fullstack_

<tf.Tensor: shape=(1, 70), dtype=float32, numpy=
array([[ 13.839393  ,   6.8539944 ,   5.931263  ,   5.7240086 ,
          0.68651676,  -0.13931084,  -1.0833197 ,  -3.8023577 ,
         -5.8675804 ,  -6.255658  ,  -7.5056405 ,  -9.337064  ,
         -9.521078  , -11.375254  , -12.07324   , -12.212617  ,
        -13.842919  , -15.976428  , -16.849522  , -18.184921  ,
        -18.589     , -20.803288  , -20.929811  , -21.30822   ,
        -22.467106  , -23.103722  , -23.871132  , -23.998785  ,
        -24.39679   , -24.599972  , -25.23382   , -27.582123  ,
        -28.04513   , -28.119186  , -28.599188  , -28.645962  ,
        -31.471348  , -31.522717  , -31.879982  , -32.140556  ,
        -32.29202   , -32.553783  , -33.462387  , -33.50419   ,
        -33.521362  , -33.731533  , -34.07652   , -34.34073   ,
        -34.37011   , -36.125423  , -38.009163  , -40.65675   ,
        -40.666573  , -43.111828  , -44.222286  , -44.96567   ,
        -45.481995  , -45.926964  , -50.921803  , -50.9

In [21]:
def scale_5(arr):
    return list(map(float, arr*5/max(arr)))

In [22]:
a = score.numpy()[0]
scale_5((a - min(a) + 1)/(max(a) - min(a)))

In [23]:
abc = list(mongo.data_3.events.aggregate([{"$match": {"userid": "User_Fresher_NodeJS_Developer_DN_15"}}, {"$group": {"_id": "$userid", "jobs": {"$addToSet": "$jobId"}}}]))[0]["jobs"]
abc

In [24]:
user_curs = list(mongo.data_3.events.aggregate([{"$match": {"jobId": {"$in": abc}}}]))
users = [i["userid"] for i in user_curs]
jobsmap = list(mongo.data_3.events.aggregate([{"$match": {"userid": {"$in": users}}}]))
jobsmap

In [25]:
import pandas as pd

In [26]:
df = pd.DataFrame(jobsmap)

In [27]:
df[df["jobId"]=="Job_Fresher_NodeJS_Developer_other_Fresher_9GC564"]

In [28]:
# _, pred = index({"user_id": np.array([np.array(["User_Fresher_NodeJS_Developer_DN_15"]), np.array(["User_Fresher_NodeJS_Developer_DN_15"])])})
# pred.numpy()
_scores, preds = index({"user_id": unique_user_ids.reshape(len(unique_user_ids), 1)})

In [29]:
dict(zip(unique_user_ids, preds.numpy()))[b"User_Fresher_Backend_Developer_DN_0"]

In [30]:
list(map(lambda x: x.decode("ascii"),preds.numpy()[0]))

In [31]:
from pymongo import UpdateOne

In [32]:
def update_data(user, pred):
    return UpdateOne({"_id": user.decode("ascii")}, {"$set": {"items": list(map(lambda x: x.decode("ascii"),pred))}}, upsert=True)

In [33]:
for u, p in zip(unique_user_ids, preds.numpy()):
    print(update_data(u, p))
    break

In [34]:
mongo.data_3.User_Recs.bulk_write(list(map(lambda u, p: update_data(u, p), unique_user_ids, preds.numpy())))

In [35]:
index({"user_id": np.array(["User_Fresher_Backend_Developer_DN_0"])})

In [36]:
mongo.data_3.User_Recs.bulk_write?